# Astroquery

Astroquery is a set of tools for querying astronomical web forms and databases.

Generally it is used to query both **Vizier** and **Simbad** catalogs from python scripts. If you have some experience in the astronomy field, you probably already know how vast is the amount of information within these catalogs.

Note these queries use Simbad/Vizier online services, so **you need a working internet connection** to run this notebook.


## Simbad catalog queries with astroquery

The easiest way to show the potential of astroquery is to show how it works:

In [ ]:
from astroquery.simbad import Simbad
result_table = Simbad.query_object("Crab Nebula")
result_table.pprint(show_unit=True)
result_table.colnames

The VOTable fields that are currently returned in the result are set to `main_id` and `coordinates`. However you can specify other fields to be added to the resulting table. To see the complete list of these fields:

In [ ]:
Simbad.list_votable_fields()

Whaaaaaaaat?! I have no idea what all these fields mean... Specially "coo_err_maja"!

In [ ]:
Simbad.get_field_description('coo_err_maja')

I see... Let's do another query on the Crab Nebula, but this time with some more "votable fields":

In [ ]:
customSimbad = Simbad()
customSimbad.add_votable_fields('otype','distance', 'velocity')

Remember to use the customized instance of Simbad for the new query:

In [ ]:
result_table = customSimbad.query_object("Crab Nebula")
result_table.pprint(show_unit=True)
result_table.colnames

In [ ]:
print("The Crab Nebula is a {} at a distance of {} {}".format(result_table['OTYPE'][0], 
                                                              result_table['distance_distance'][0], 
                                                              result_table['distance_unit'][0]))

Astroquery allows more complex queries, as for example a region around a sky coordinate:

In [ ]:
from astropy.coordinates import SkyCoord
crab_coord = SkyCoord.from_name('Crab Nebula')  
result_table = customSimbad.query_region(crab_coord, radius='2d0m0s')
result_table.pprint()

This is already too much information. By using pandas, we can easily take a peek at what we queried.

You probably won't need it, but remember you can take a look at the definition of the different Simbad otypes [here](http://simbad.u-strasbg.fr/simbad/sim-display?data=otypes).

In [ ]:
pandas_table = result_table.to_pandas()
pandas_table['OTYPE'].value_counts()

## Vizier catalog queries with astroquery

Similarly as described above, astroquery can be used to query within any Vizier catalog.

The possibilities are enormous, here just some examples are listed:


#### Query an object

We can search over the huge amount of catalogs within Vizier for a specific object:

In [ ]:
# The crab pulsar is too famous! 225 catalogs contain information of the CP...
# result = Vizier.query_object("Crab pulsar")

result = Vizier.query_object("FRB121102")
print(result)

In [ ]:
result[0] # Equivalent to result['I/197A/tic']

#### Finding catalogs by keyword

You can also search through the Vizier catalogs through keywords (e. g. authors, title, etc...):

In [ ]:
from astroquery.vizier import Vizier
# Let's try to find some Fermi-LAT catalogs.

# This query is probably too broad... 391 available catalogs with the keyword "gamma"
# catalog_list = Vizier.find_catalogs('Gamma', max_catalogs=10000)

# Fewer results if we directly search for the author name:
catalog_list = Vizier.find_catalogs('Abdo', max_catalogs=10000)
print({k:v.description for k,v in catalog_list.items()})

Let's take a look at the 2nd Fermi-LAT catalog of gamma-ray pulsars (key = 'J/ApJS/208/17')

In [ ]:
catalog = Vizier.get_catalogs('J/ApJS/208/17')

In [ ]:
catalog.pprint()

In [ ]:
catalog[0]

#### Query a region using Vizier catalogs

Similar as with Simbad, let's do a query around a sky region:

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u

# Select only bright stars
v = Vizier(column_filters={"Bmag":"<9"})
v.ROW_LIMIT = -1
crabNebula = SkyCoord.from_name('Crab Nebula')  
result = v.query_region(crabNebula, radius=2.0*u.deg, catalog='NOMAD')
result[0]

In [ ]:
nomad_query = result[0]
nomad_query['RAJ2000', 'DEJ2000','Bmag']

Classic problem: convert ra/dec into alt/az, and see which star is bothering you within your field of view:

In [ ]:
from astropy import coordinates as c
from astropy.coordinates import SkyCoord
from astropy import time as t

magicSite = c.EarthLocation( lat=28.76194399284443*u.deg, lon=-17.890066533603996*u.deg )
star_coord = SkyCoord(ra=nomad_query['RAJ2000'], dec=nomad_query['DEJ2000'], frame='icrs')
star_coord_altAz = star_coord.transform_to( c.AltAz(obstime=t.Time.now(), location=magicSite) )
nomad_query['alt'] = star_coord_altAz.alt
nomad_query['az'] = star_coord_altAz.az

In [ ]:
nomad_query.sort('Bmag')
nomad_query['Bmag', 'alt', 'az'].pprint()

### Resources:

* [Astroquery documentation](http://astroquery.readthedocs.io/en/latest/)
* [Astroquery Vizier docs](http://astroquery.readthedocs.io/en/latest/vizier/vizier.html)
* [Astroquery Simbad docs](http://astroquery.readthedocs.io/en/latest/simbad/simbad.html)
* [Simple examples](http://astroquery.readthedocs.io/en/latest/gallery.html)
* [The Naval Observatory Merged Astrometric Dataset (NOMAD)](http://www.usno.navy.mil/USNO/astrometry/optical-IR-prod/nomad)
